In [ ]:
import math
import cmath

EPS = 0.99
THETA0 = math.pi / 4
K = 120

TOL_AB = 1e-8
TOL_M  = 1e-8
Q_MIN  = 1e-14
BLOWUP = 1e9

M_SAMPLES = 61
MIN_LEN = 1.20
NEAR_ENDPOINT = 5e-3

def rg_localized_by_K(eps: float, omega: float, K: int) -> bool:
    """
    Implements (4)-(6) with safety checks (8), and tests (7).
    Returns True iff omega is RG-localized by depth K.
    """
    z = cmath.exp(1j * omega)
    zi = 1.0 / z

    a = 1.0 + 0j
    b = 1.0 + 0j
    m = 0.0 + 0j

    for k in range(K + 1):
        theta = THETA0 * (eps ** k)
        s = math.sin(theta)
        c = math.cos(theta)

        Q = (zi * zi) * (s * s) + (zi * c - m) ** 2

        # resonance / blowup guards
        if abs(Q) < Q_MIN:
            return False
        if abs(a) > BLOWUP or abs(b) > BLOWUP or abs(m) > BLOWUP:
            return False

        # localization test at this depth
        if (abs(a) + abs(b) <= TOL_AB) and (abs(abs(m) - 1.0) <= TOL_M):
            return True

        if k == K:
            break

        # compute next values from current (do NOT update in-place prematurely)
        inv11 = (zi * s) / Q
        inv22 = (-zi * s) / Q
        inv12 = (zi * c - m) / Q

        a_next = (a * a) * inv11
        b_next = (b * b) * inv22
        m_next = m + (a * b) * inv12

        a, b, m = a_next, b_next, m_next

    return False

def verify_output(triple):
    # parse
    if (not isinstance(triple, (tuple, list))) or len(triple) != 3:
        return False, "answer() must return a tuple/list of length 3: (omega_lo, omega_hi, omega_res)."

    omega_lo, omega_hi, omega_res = triple

    # type checks
    for name, x in [("omega_lo", omega_lo), ("omega_hi", omega_hi), ("omega_res", omega_res)]:
        if not isinstance(x, (int, float)):
            return False, f"{name} must be a real number (int/float)."

    # basic constraints
    if not (0.0 < omega_lo < omega_hi < math.pi):
        return False, "Need 0 < omega_lo < omega_hi < pi."
    if (omega_hi - omega_lo) < MIN_LEN:
        return False, f"Interval too short: need omega_hi - omega_lo >= {MIN_LEN}."

    if not (0.0 <= omega_res <= math.pi):
        return False, "Need 0 <= omega_res <= pi."
    if omega_lo <= omega_res <= omega_hi:
        return False, "omega_res must lie OUTSIDE [omega_lo, omega_hi]."

    # endpoint proximity constraint
    if min(abs(omega_res - omega_lo), abs(omega_res - omega_hi)) > NEAR_ENDPOINT:
        return False, f"omega_res must be within {NEAR_ENDPOINT} of omega_lo or omega_hi."

    # check localization on sampled grid in the interval
    for j in range(M_SAMPLES):
        t = j / (M_SAMPLES - 1)
        omega = omega_lo + t * (omega_hi - omega_lo)
        if not rg_localized_by_K(EPS, omega, K):
            return False, f"Interval sample failed localization at omega={omega:.12f} (j={j})."

    # check resonance witness
    if rg_localized_by_K(EPS, omega_res, K):
        return False, f"omega_res={omega_res:.12f} is localized, but must be non-localized/resonant."

    return True, "PASS"

# Optional CLI harness:
if __name__ == "__main__":
    # Example usage:
    #   from submission import answer
    #   ok, msg = verify_output(answer())
    #   print(ok, msg)
    print("Verifier ready. Import verify_output(...) and rg_localized_by_K(...).")2Evscode-notebook-cell:/Users/richa/Noether/Emily_envs/Challenge1.ipynb